In [1]:
####################### AUTO LOGIN #####################
import os
os.system('clear')
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
print('Starting Strategy...')
import json
import login
from login import *
import pandas as pd
import joblib
import ta
import requests
import numpy as np
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
import h2o
hh=h2o.init(ip='127.0.0.1',port='12349')
n50 = pd.read_csv('n50.csv')
nfifty_tokens_list = n50['instrument_token'].to_numpy().tolist()
n100 = pd.read_csv('n100.csv')
n100_tokens_list = n100['instrument_token'].to_numpy().tolist()

Starting Strategy...
access token : oas2EvdTklb7NoI1Nhpl1VIjGN5c1TVu
request token : DvX5QyOsCHAfd9TFzVqPApk6sBAhNdBh


Loading BokehJS ...

Checking whether there is an H2O instance running at http://127.0.0.1:12349 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "14.0.2" 2020-07-14; OpenJDK Runtime Environment (build 14.0.2+12-Ubuntu-2); OpenJDK 64-Bit Server VM (build 14.0.2+12-Ubuntu-2, mixed mode, sharing)
  Starting server from /home/hemang/.local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwd_ovbwk
  JVM stdout: /tmp/tmpwd_ovbwk/h2o_hemang_started_from_python.out
  JVM stderr: /tmp/tmpwd_ovbwk/h2o_hemang_started_from_python.err
  Server is running at http://127.0.0.1:12349
Connecting to H2O server at http://127.0.0.1:12349 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,"28 days, 3 hours and 30 minutes"
H2O_cluster_name:,H2O_from_python_hemang_nqblip
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.848 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [12]:
order_type = 'MIS' 
mymargin = 35000
interval='5minute'
daysdiff = 10
bkg = 0.0
isneg = 1

todaydt=datetime.date.today()
hud_ago=todaydt-datetime.timedelta(days=daysdiff)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)

print('from_date :'+str(from_date))
print('to_date :'+str(to_date))

from_date :2021-01-11
to_date :2021-01-21


In [41]:
# one stock predict and place order

saved_model = h2o.load_model('../XGBoost_2_AutoML_20210109_204532')
saved_model2 = h2o.load_model('../GBM_grid__1_AutoML_20210109_115241_model_1')
saved_model3 = h2o.load_model('../DRF_1_AutoML_20210111_165257')
saved_model4 = h2o.load_model('../GBM_3_AutoML_20210113_220248')
saved_model5 = h2o.load_model('../GBM_grid__1_AutoML_20210120_200251_model_13')

def one_stk_fun(insttkn,tradeon=False):
    '''one_stk_fun(insttkn,tradeon=False)'''
    global isneg
    old_lst = kite.historical_data(insttkn, from_date, to_date, interval,continuous=False)

    mydf = pd.DataFrame(old_lst)
    mydf = mydf.fillna(0)
    mydf = ta.add_all_ta_features(mydf, open="open", high="high", low="low", close="close", volume="volume")
    mydf = mydf.fillna(0)[150:]
    
    hf = h2o.H2OFrame(mydf)

    prd1 = saved_model.predict(hf)
    ct1 = prd1.concat([hf])
    prd2 = saved_model2.predict(ct1)
    ct2 = prd2.concat([ct1])
    prd3 = saved_model3.predict(ct2)
    
    
    cls = ct2.col_names
    
    for n, i in enumerate(cls):
        if (i == 'predict0'):
            cls[n] = 'predict0.1'
        if (i == 'predict'):
            cls[n] = 'predict0'
        if (i == 'p-1'):
            cls[n] = 'p-10'
        if (i == 'p0'):
            cls[n] = 'p00'
        if (i == 'p1'):
            cls[n] = 'p10'
            
    ct2.set_names(cls)
    
    ct3 = prd3.concat([ct2])
    prd4 = saved_model4.predict(ct3)
    ct4 = prd4.concat([ct3])
    
    ipf = h2o.as_list(ct4)  

    ipf['predict'] = ipf['predict']-ipf['close']
    ipf['predict0.1'] = ipf['predict0.1']-ipf['close']
    ipf['next_close'] = ipf['close'].shift(periods=-1)
    ipf['next_close'] = ipf['next_close']-ipf['close']

    ipf['date'] = pd.to_datetime(ipf['date'])

    ipf['year'] = ipf['date'].dt.year
    ipf['month'] = ipf['date'].dt.month
    ipf['week'] = ipf['date'].dt.isocalendar().week
    ipf['day'] = ipf['date'].dt.day
    ipf['hour'] = ipf['date'].dt.hour
    ipf['minute'] = ipf['date'].dt.minute
    ipf['dayofweek'] = ipf['date'].dt.dayofweek
    ipf['weekday'] = ipf['date'].dt.weekday
    ipf['quarter'] = ipf['date'].dt.quarter
    ipf['is_month_start'] = ipf['date'].dt.is_month_start
    ipf['is_month_end'] = ipf['date'].dt.is_month_end
    ipf['is_quarter_start'] = ipf['date'].dt.is_quarter_start
    ipf['is_quarter_end'] = ipf['date'].dt.is_quarter_end
    ipf['is_year_start'] = ipf['date'].dt.is_year_start
    ipf['is_year_end'] = ipf['date'].dt.is_year_end
    ipf['is_leap_year'] = ipf['date'].dt.is_leap_year
    ipf['daysinmonth'] = ipf['date'].dt.daysinmonth
    ipf['month_name'] = ipf['date'].dt.month_name()
    ipf['day_name'] = ipf['date'].dt.day_name()

    ipf['hl_range'] = ipf['high']-ipf['low']
    ipf['oc_range'] = ipf['open']-ipf['close']
    ipf['ol_range'] = ipf['open']-ipf['low']
    ipf['oh_range'] = ipf['open']-ipf['high']
    ipf['cl_range'] = ipf['close']-ipf['low']
    ipf['ch_range'] = ipf['close']-ipf['high']

    ipf_h2o_df = h2o.H2OFrame(ipf)
#     print(ipf_h2o_df)

    prd5 = saved_model5.predict(ipf_h2o_df)
    ct5 = prd5.concat([ipf_h2o_df])
    ct5

    os.system('clear')
    data_as_df = h2o.as_list(ct5)  

    sel_data = data_as_df[['predict','predict0','predict0.1','date','close','next_close']] #,'next_close'

    last_isbullbear = 0

    total_gained_pt = []
    total_gained_pt_total = 0
    total_gained_pt_lst = []

    for ind in sel_data.index:
        
        is_bullbear = 0
        if(ind<(len(sel_data.index)-1)):
            if(sel_data['next_close'][ind]>0):
                is_bullbear=1
            elif(sel_data['next_close'][ind]<0):
                is_bullbear=-1
                
        pred_bullbear = 0
        if(ind<(len(sel_data.index)-1)):
            if(sel_data['predict'][ind]>0):
                pred_bullbear=1
            elif(sel_data['predict'][ind]<0):
                pred_bullbear=-1
        pred_true = -1

        if(pred_bullbear==is_bullbear):
            pred_true = 1

        gained_pt=0
        if(ind<(len(sel_data.index)-1)):
            gained_pt = pred_true*abs(sel_data['next_close'][ind])

            
        is_global_avail = 'isneg'+str(insttkn) in globals() 
        if(is_global_avail):
            gained_pt = globals()['isneg'+str(insttkn)]*gained_pt
            isneg=globals()['isneg'+str(insttkn)]
        else:
            gained_pt = gained_pt*isneg

        if(ind>0 and (last_isbullbear!=is_bullbear)):
            gained_pt = gained_pt+bkg

        total_gained_pt.append(gained_pt)
        total_gained_pt_total = total_gained_pt_total + gained_pt
        total_gained_pt_lst.append(total_gained_pt_total)
        last_isbullbear = is_bullbear
    
    this_mean_val = np.array(total_gained_pt_lst).mean()
    gdptdf = pd.DataFrame({'total_gained_pt':total_gained_pt,'total_gained_pt_lst':total_gained_pt_lst})
    gdptdf.plot()

    nw = datetime.datetime.now()
    last_candle_time =  nw.replace(hour=15, minute=30, second=0, microsecond=0)
    first_candle_time =  nw.replace(hour=9, minute=00, second=0, microsecond=0)
    zero_hour =  nw.replace(hour=0, minute=0, second=0, microsecond=0)
    
    selected_time_str=''
    
    if(nw.minute<5):
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'{0}'.format(str(nw.hour-1).zfill(2))+':' +'{0}'.format(str(55+nw.minute).zfill(2))+':'+'00+05:30'
    else:
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'{0}'.format(str(nw.hour).zfill(2))+':' +'{0}'.format(str(nw.minute-5).zfill(2))+':'+'00+05:30'
    if(nw<last_candle_time):
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
    if(nw>first_candle_time):
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day-1).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
    
    this_qt = sel_data.loc[sel_data['date'] == selected_time_str].to_numpy()[0]

    pred_close = this_qt[0]
    this_close = this_qt[4]
    this_time = this_qt[3]

    this_mean_percent = (this_mean_val/this_close)*100
    
    is_global_avail = 'isneg'+str(insttkn) in globals() 
    if(is_global_avail):
        globals()['isneg'+str(insttkn)]=globals()['isneg'+str(insttkn)]
    else:
        if(this_mean_percent<0):
            globals()['isneg'+str(insttkn)]=-1
        else:
            globals()['isneg'+str(insttkn)]=1

    current_bullbear = 0

    if(pred_close>0):
        current_bullbear=1
    elif(pred_close<0):
        current_bullbear=-1

    print('=============================')
    print('current_bullbear : '+str(current_bullbear))
    print('pred_close : '+str(pred_close))
    print('this_close : '+str(this_close))
    print('this_mean_percent : '+str(this_mean_percent))
    print('this_time : '+str(this_time))
    print('=============================')

    if(tradeon):
        is_global_avail = 'isneg'+str(insttkn) in globals() 
        this_neg_local = 0
        if(is_global_avail):
            this_neg_local = globals()['isneg'+str(insttkn)]
        else:
            this_neg_local = isneg
        print('isneg : '+str(this_neg_local))
        final_qt = int(this_neg_local*current_bullbear*full_qty)
        
        print('Placing order for : '+str(insttkn)+' : qty : '+str(final_qt))
        ckqnt(0,final_qt)

    globals()['df'+str(insttkn)] = pd.DataFrame({insttkn:total_gained_pt_lst})
    globals()['df'+str(insttkn)] = (globals()['df'+str(insttkn)]/this_close)*100
    return globals()['df'+str(insttkn)]

In [42]:
# Select Target Token and Symbol Name
def decide_target(nfifty_tokens_list):
    global mynp,last_mean
    '''decide_target(nfifty_tokens_list)'''
    combined_df = pd.DataFrame()
    counter1 = 1
    for iii in nfifty_tokens_list:
        
        symbol_name1 = n50.loc[n50['instrument_token'] == iii, 'tradingsymbol'].to_numpy()[0]
        print()
        print('============================================================================')
        print('Analyzing Token : '+str(iii)+ ' : '+str(symbol_name1)+ ' : '+str(counter1)+'/50')
        print('============================================================================')
        counter1 = counter1+1
        one_stk_fun(iii,tradeon=False)
        combined_df = pd.concat([combined_df,globals()['df'+str(iii)]], axis=1)

    combined_df.plot()
    os.system('clear')
    mynp = combined_df.iloc[[-1]].to_numpy()[0]
    mynp = np.nan_to_num(mynp)
    print('Nifty50 Mean : '+str(np.mean(mynp)))
    print('Nifty50 Sum : '+str(np.sum(mynp)))

    insttkn = 0
    last_mean = 0

    for ij in range(len(nfifty_tokens_list)):
        this_mean = combined_df[nfifty_tokens_list[ij]].to_numpy().mean()

        if(this_mean>last_mean):
            insttkn= nfifty_tokens_list[ij]
            last_mean = this_mean

    symbol_ip = n50.loc[n50['instrument_token'] == insttkn, 'tradingsymbol'].to_numpy()[0]

    print('Success Rate Mean : '+str(last_mean))
    print('Selected Token : '+str(insttkn))
    print('Selected Symbol : '+str(symbol_ip))
    return {'insttkn':insttkn,'symbol_ip':symbol_ip,'mean':last_mean}

In [43]:
# base functions : getquant, placeneworder, ckqnt

myquantity = 0
def getquant():
    global myquantity,order_type,symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))

stop_trading = False
stopbuy = False
stopsell = False
ex_type = kite.EXCHANGE_NSE

def placeneworder(quantdiff,price_ip):
    global stopbuy,stop_trading,stopsell,order_type,symbol_ip,ex_type
    if(quantdiff>0 and stopbuy == False and stop_trading==False):
        try:
            stop_trading = True
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=ex_type,
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
            stopsell = False
            stop_trading = False
        except Exception as e:
#             stopbuy = True
            print(e)
            getquant()
    if(quantdiff<0 and stopsell == False and stop_trading==False):
        try:
            stop_trading = True
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=ex_type,
                        transaction_type=kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
            stopbuy = False
            stop_trading = False
        except Exception as e:
#             stopsell = True
            print(e)
            getquant()

def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff,orderprice)
    return myquantity

def print_params():
    print('=============================')
    print('Nifty50 Mean : '+str(np.mean(mynp)))
    print('Nifty50 Sum : '+str(np.sum(mynp)))
    print('Success Rate Mean : '+str(last_mean))
    print('Selected Token : '+str(insttkn))
    print('Selected Symbol : '+str(symbol_ip))
    print('Order Type : '+str(order_type))
    print('My Margin : '+str(mymargin))
    print('Interval : '+str(interval))
    print('MIS Qty : '+str(full_qty))
    print('=============================')

In [44]:
############### Calculate MIS Qty ############

url = 'https://api.kite.trade/margins/equity'
margin_data = requests.get(url, timeout=2.50)
margin_map = json.loads(margin_data.content.decode('utf-8'))
margin_map_df = pd.DataFrame(margin_map)
margin_map_df

ipmap = decide_target(nfifty_tokens_list)
ipmap = decide_target(nfifty_tokens_list)

symbol_ip = ipmap['symbol_ip']
insttkn = ipmap['insttkn']

mis_margin = margin_map_df.loc[margin_map_df['tradingsymbol'] == symbol_ip, 'mis_multiplier'].to_numpy()[0]

myqt = kite.quote([insttkn])
ltp = myqt[str(insttkn)]['last_price']
mis_margin_ip = ltp/mis_margin

full_qty = int(mymargin/mis_margin_ip)
full_qty

print('Selected Token :'+str(insttkn))
print('Selected Symbol :'+str(symbol_ip))
print('Selected full_qty :'+str(full_qty))
print('Selected isneg :'+str(globals()['isneg'+str(insttkn)]))


Analyzing Token : 3861249 : ADANIPORTS : 1/50
Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/hemang/.local/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'month_name' has levels not trained on: ["January"]
  warnings.warn(w)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
# 5 minute time loop with predict/order Fn
import time
last_min = 0
getquant()
now = datetime.datetime.now()
start_time = now.replace(hour=10, minute=0, second=0, microsecond=0)
stop_time = now.replace(hour=14, minute=45, second=0, microsecond=0)
now = datetime.datetime.now()
while(1):
    minute=datetime.datetime.now().minute
    now = datetime.datetime.now()
    if((start_time < now ) and (stop_time > now)):
        if(minute%5==0 and(minute!=last_min)):
            os.system('clear')
            one_stk_fun(insttkn,True)
            last_min=minute
        time.sleep(0.1)
    else:
        symbol_name2 = n50.loc[n50['instrument_token'] == insttkn, 'tradingsymbol'].to_numpy()[0]
        ckqnt(0,0)
        time.sleep(0.5)
        print('out of time...!')

In [8]:
# one_stk_fun(1270529,tradeon=False)
# # 969473
# # 81153

In [9]:
# import py_compile
# py_compile.compile('./mystrat.py')

In [10]:
# ipynb-py-convert stock_predict_pipe_11_1_2021.ipynb mystrat.py
# python3 -m compileall mystrat.py

In [36]:
todaydt=datetime.date.today()

for i in range(2):
    
    to_proloss_date=todaydt-datetime.timedelta(days=i)
    from_proloss_date=todaydt-datetime.timedelta(days=i+4)
    to_proloss_date=datetime.date.isoformat(to_proloss_date)
    from_proloss_date=datetime.date.isoformat(from_proloss_date)
    print('to_proloss_date '+str(to_proloss_date))
    print('from_proloss_date '+str(from_proloss_date))
    
######################################    

    to_calc_taget_date=todaydt-datetime.timedelta(days=i+5)
    from_calc_taget_date=todaydt-datetime.timedelta(days=i+9)
    to_calc_taget_date=datetime.date.isoformat(to_calc_taget_date)
    from_calc_taget_date=datetime.date.isoformat(from_calc_taget_date)
    print('to_calc_taget_date '+str(to_calc_taget_date))
    print('from_calc_taget_date '+str(from_calc_taget_date))

################################    
    
    to_date = to_calc_taget_date
    from_date = from_calc_taget_date
    map1 = decide_target(nfifty_tokens_list)
    
#################################

    to_date = to_proloss_date
    from_date = from_proloss_date
    
    tknip = map1['insttkn']
    print('*********************************************')
    one_stk_fun(tknip,False)
    print('*********************************************')
    
    print()    

to_proloss_date 2021-01-21
from_proloss_date 2021-01-17
to_calc_taget_date 2021-01-16
from_calc_taget_date 2021-01-12

Analyzing Token : 3861249 : ADANIPORTS : 1/50
Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/hemang/.local/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'month_name' has levels not trained on: ["January"]
  warnings.warn(w)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [40]:
# # one stock predict and place order

# saved_model = h2o.load_model('../XGBoost_2_AutoML_20210109_204532')
# saved_model2 = h2o.load_model('../GBM_grid__1_AutoML_20210109_115241_model_1')
# saved_model3 = h2o.load_model('../DRF_1_AutoML_20210111_165257')
# saved_model4 = h2o.load_model('../GBM_3_AutoML_20210113_220248')
# saved_model5 = h2o.load_model('../GBM_grid__1_AutoML_20210120_200251_model_13')

# def one_stk_fun(insttkn,tradeon=False):
#     '''one_stk_fun(insttkn,tradeon=False)'''
#     global isneg
insttkn = 1270529
tradeon = False
old_lst = kite.historical_data(insttkn, from_date, to_date, interval,continuous=False)

mydf = pd.DataFrame(old_lst)
mydf = mydf.fillna(0)
mydf = ta.add_all_ta_features(mydf, open="open", high="high", low="low", close="close", volume="volume")
mydf = mydf.fillna(0)[150:]

hf = h2o.H2OFrame(mydf)

prd1 = saved_model.predict(hf)
ct1 = prd1.concat([hf])
prd2 = saved_model2.predict(ct1)
ct2 = prd2.concat([ct1])
prd3 = saved_model3.predict(ct2)


cls = ct2.col_names

for n, i in enumerate(cls):
    if (i == 'predict0'):
        cls[n] = 'predict0.1'
    if (i == 'predict'):
        cls[n] = 'predict0'
    if (i == 'p-1'):
        cls[n] = 'p-10'
    if (i == 'p0'):
        cls[n] = 'p00'
    if (i == 'p1'):
        cls[n] = 'p10'

ct2.set_names(cls)

ct3 = prd3.concat([ct2])
prd4 = saved_model4.predict(ct3)
ct4 = prd4.concat([ct3])

ipf = h2o.as_list(ct4)  

ipf['predict'] = ipf['predict']-ipf['close']
ipf['predict0.1'] = ipf['predict0.1']-ipf['close']
ipf['next_close'] = ipf['close'].shift(periods=-1)
ipf['next_close'] = ipf['next_close']-ipf['close']

ipf['date'] = pd.to_datetime(ipf['date'])

ipf['year'] = ipf['date'].dt.year
ipf['month'] = ipf['date'].dt.month
ipf['week'] = ipf['date'].dt.isocalendar().week
ipf['day'] = ipf['date'].dt.day
ipf['hour'] = ipf['date'].dt.hour
ipf['minute'] = ipf['date'].dt.minute
ipf['dayofweek'] = ipf['date'].dt.dayofweek
ipf['weekday'] = ipf['date'].dt.weekday
ipf['quarter'] = ipf['date'].dt.quarter
ipf['is_month_start'] = ipf['date'].dt.is_month_start
ipf['is_month_end'] = ipf['date'].dt.is_month_end
ipf['is_quarter_start'] = ipf['date'].dt.is_quarter_start
ipf['is_quarter_end'] = ipf['date'].dt.is_quarter_end
ipf['is_year_start'] = ipf['date'].dt.is_year_start
ipf['is_year_end'] = ipf['date'].dt.is_year_end
ipf['is_leap_year'] = ipf['date'].dt.is_leap_year
ipf['daysinmonth'] = ipf['date'].dt.daysinmonth
ipf['month_name'] = ipf['date'].dt.month_name()
ipf['day_name'] = ipf['date'].dt.day_name()

ipf['hl_range'] = ipf['high']-ipf['low']
ipf['oc_range'] = ipf['open']-ipf['close']
ipf['ol_range'] = ipf['open']-ipf['low']
ipf['oh_range'] = ipf['open']-ipf['high']
ipf['cl_range'] = ipf['close']-ipf['low']
ipf['ch_range'] = ipf['close']-ipf['high']

ipf_h2o_df = h2o.H2OFrame(ipf)
#     print(ipf_h2o_df)

prd5 = saved_model5.predict(ipf_h2o_df)
ct5 = prd5.concat([ipf_h2o_df])
ct5

os.system('clear')
data_as_df = h2o.as_list(ct5)  

sel_data = data_as_df[['predict','predict0','predict0.1','date','close','next_close']] #,'next_close'

last_isbullbear = 0

total_gained_pt = []
total_gained_pt_total = 0
total_gained_pt_lst = []

for ind in sel_data.index:

    is_bullbear = 0
    if(ind<(len(sel_data.index)-1)):
        if(sel_data['next_close'][ind]>0):
            is_bullbear=1
        elif(sel_data['next_close'][ind]<0):
            is_bullbear=-1

    pred_bullbear = 0
    if(ind<(len(sel_data.index)-1)):
        if(sel_data['predict'][ind]>0):
            pred_bullbear=1
        elif(sel_data['predict'][ind]<0):
            pred_bullbear=-1
    pred_true = -1

    if(pred_bullbear==is_bullbear):
        pred_true = 1

    gained_pt=0
    if(ind<(len(sel_data.index)-1)):
        gained_pt = pred_true*abs(sel_data['next_close'][ind])


    is_global_avail = 'isneg'+str(insttkn) in globals() 
    if(is_global_avail):
        gained_pt = globals()['isneg'+str(insttkn)]*gained_pt
        isneg=globals()['isneg'+str(insttkn)]
    else:
        gained_pt = gained_pt*isneg

    if(ind>0 and (last_isbullbear!=is_bullbear)):
        gained_pt = gained_pt+bkg

    total_gained_pt.append(gained_pt)
    total_gained_pt_total = total_gained_pt_total + gained_pt
    total_gained_pt_lst.append(total_gained_pt_total)
    last_isbullbear = is_bullbear

this_mean_val = np.array(total_gained_pt_lst).mean()
gdptdf = pd.DataFrame({'total_gained_pt':total_gained_pt,'total_gained_pt_lst':total_gained_pt_lst})
gdptdf.plot()

nw = datetime.datetime.now()
last_candle_time =  nw.replace(hour=15, minute=30, second=0, microsecond=0)
first_candle_time =  nw.replace(hour=9, minute=00, second=0, microsecond=0)
zero_hour =  nw.replace(hour=0, minute=0, second=0, microsecond=0)

selected_time_str = sel_data['date'][-1:].to_numpy()[0]
print(type(selected_time_str))
print(type(sel_data['date'][-1:].to_numpy()[0]))
print(sel_data)


#     selected_time_str=''

#     if(nw.minute<5):
#         selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'{0}'.format(str(nw.hour-1).zfill(2))+':' +'{0}'.format(str(55+nw.minute).zfill(2))+':'+'00+05:30'
#     else:
#         selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'{0}'.format(str(nw.hour).zfill(2))+':' +'{0}'.format(str(nw.minute-5).zfill(2))+':'+'00+05:30'

# #     if(nw<last_candle_time):
# #         selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
# #     if(nw>first_candle_time):
# #         selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day-1).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'

if(nw<last_candle_time):
    selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
if(nw>first_candle_time):
    selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day-1).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'

this_qt = sel_data.loc[sel_data['date'] == selected_time_str].to_numpy()[0]

pred_close = this_qt[0]
this_close = this_qt[4]
this_time = this_qt[3]

this_mean_percent = (this_mean_val/this_close)*100

is_global_avail = 'isneg'+str(insttkn) in globals() 
if(is_global_avail):
    globals()['isneg'+str(insttkn)]=globals()['isneg'+str(insttkn)]
else:
    if(this_mean_percent<0):
        globals()['isneg'+str(insttkn)]=-1
    else:
        globals()['isneg'+str(insttkn)]=1

current_bullbear = 0

if(pred_close>0):
    current_bullbear=1
elif(pred_close<0):
    current_bullbear=-1

print('=============================')
print('current_bullbear : '+str(current_bullbear))
print('pred_close : '+str(pred_close))
print('this_close : '+str(this_close))
print('this_mean_percent : '+str(this_mean_percent))
print('this_time : '+str(this_time))
print('=============================')

if(tradeon):
    is_global_avail = 'isneg'+str(insttkn) in globals() 
    this_neg_local = 0
    if(is_global_avail):
        this_neg_local = globals()['isneg'+str(insttkn)]
    else:
        this_neg_local = isneg
    print('isneg : '+str(this_neg_local))
    final_qt = int(this_neg_local*current_bullbear*full_qty)

    print('Placing order for : '+str(insttkn)+' : qty : '+str(final_qt))
    ckqnt(0,final_qt)

globals()['df'+str(insttkn)] = pd.DataFrame({insttkn:total_gained_pt_lst})
globals()['df'+str(insttkn)] = (globals()['df'+str(insttkn)]/this_close)*100
#     return globals()['df'+str(insttkn)]

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/hemang/.local/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'month_name' has levels not trained on: ["January"]
  warnings.warn(w)


<class 'str'>
<class 'str'>
        predict  predict0  predict0.1                       date   close  \
0      0.535511  0.305270          -1  2021-01-14 09:15:00+05:30  556.50   
1      1.244281  0.726911          -1  2021-01-14 09:20:00+05:30  556.00   
2     10.200030  1.234076          -1  2021-01-14 09:25:00+05:30  556.50   
3     10.288307  1.382626           1  2021-01-14 09:30:00+05:30  554.40   
4     10.109643  0.881864          -1  2021-01-14 09:35:00+05:30  554.95   
..          ...       ...         ...                        ...     ...   
145  -57.801305  0.805185           1  2021-01-15 15:05:00+05:30  544.55   
146   -6.661726  1.137036          -1  2021-01-15 15:10:00+05:30  543.50   
147 -111.118260  1.187526           1  2021-01-15 15:15:00+05:30  542.40   
148 -110.918312 -0.809124           1  2021-01-15 15:20:00+05:30  542.30   
149 -110.681300 -0.399641           1  2021-01-15 15:25:00+05:30  541.95   

     next_close  
0         -0.50  
1          0.50  
2    

IndexError: index 0 is out of bounds for axis 0 with size 0